# OTX API DEMO (MMCLS Example)

## Customization Training API

Select a framework & import adapter modules.

We'll choose MMCLS here, and we'll import the following modules.

In [1]:
from otx.v2.adapters.torch.mmcv.mmcls import Dataset, build_model_from_config
from otx.v2.adapters.torch.mmcv.mmcls.engine import MMCLSEngine  # TODO: Import structure that needs to be changed
# or from otx.adapters.torch.mmcv.mmcls import *

/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/openvino/pyopenvino/__init__.py:10: FutureWarning: The module is private and following namespace `pyopenvino` will be removed in the future
  warnings.warn(message="The 

## Prepare Dataset & DataLoader
1. Prepare a dataset and enter path into Dataset

    - Convert to OTX's DatasetEntity and Label Schema by leveraging Datumaro's features through paths (path -> Datumaro -> OTX DatasetEntity & LabelSchema)

In [2]:
dataset = Dataset(
    train_data_roots="../../../../tests/assets/classification_dataset_class_incremental",
    val_data_roots="../../../../tests/assets/classification_dataset_class_incremental",
    test_data_roots="../../../../tests/assets/classification_dataset_class_incremental",
)

2-1. Build Torch Dataset from MMCV config (filepath or dict) -> torch.utils.data.Dataset

    - User can build a dataset from a config file or dictionary used by MMCV.

In [4]:
train_dataloader = dataset.train_dataloader()
print(f"Dataset type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")
print(f"Number of classes: {dataset.num_classes}")

Dataset type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 32
Dataset size: 32
Number of classes: 3


In [5]:
# Customize batch_size
train_dataloader = dataset.train_dataloader(batch_size=2)
print(f"DataLoader type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")
print(f"Number of classes: {train_dataloader.dataset.num_classes}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 16
Dataset size: 32
Number of classes: 3


In [6]:
# With Config File
data_config = "/home/harimkan/workspace/repo/otx-fork-3/src/otx/algorithms/classification/configs/efficientnet_b0_cls_incr/data_pipeline.py"

train_dataloader = dataset.train_dataloader(
    config=data_config,
    batch_size=1
)
print(f"DataLoader type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")
print(f"Number of classes: {train_dataloader.dataset.num_classes}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 32
Dataset size: 32
Number of classes: 3


2-2. Or Build a custom dataset using mmcv's Data pipeline API -> torch.utils.data.Dataset

    - User can build the data pipeline by configuring it using mmcv's API, without using Config.

In [7]:
# Customize mmcls pipeline
# Build using pipeline Object
from mmcls.datasets.pipelines import Resize, Normalize, ImageToTensor, Collect, ToTensor
data_pipeline = [
    Resize(size=224),
    Normalize(mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True),
    ImageToTensor(keys=["img"]),
    ToTensor(keys=["gt_label"]),
    Collect(keys=["img", "gt_label"])
]
train_dataloader_2 = dataset.train_dataloader(
    pipeline=data_pipeline
)
print(f"DataLoader type: {type(train_dataloader_2)}")
print(f"Length of DataLoader: {len(train_dataloader_2)}")
print(f"Dataset size: {len(train_dataloader_2.dataset)}")
print(f"Number of classes: {train_dataloader_2.dataset.num_classes}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 32
Dataset size: 32
Number of classes: 3


## Prepare Model
Config to build the torch.nn.Module model. -> Provide function for building models so that each framework's config can be used

    - Users can build a torch.nn.Module via config as well

In [8]:
# Build from config file
model = build_model_from_config(
    config="/home/harimkan/workspace/repo/otx-fork-3/src/otx/v2/configs/classification/mmcls_efficientnet_b0.yaml",
    num_classes=dataset.num_classes
)
print(f"Model type: {type(model)}")
for i, data_batch in enumerate(train_dataloader):
    output = model.train_step(data_batch)
    print(output)

2023-07-05 13:26:35,684 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-07-05 13:26:35,708 | INFO : 'in_channels' config in model.head is updated from -1 to 1280
2023-07-05 13:26:35,767 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip


Model type: <class 'otx.v2.adapters.torch.mmcv.mmcls.modules.models.classifiers.sam_classifier.SAMImageClassifier'>
{'loss': tensor(0.8474, grad_fn=<AddBackward0>), 'log_vars': OrderedDict([('loss', 0.8473533987998962)]), 'num_samples': 1}
{'loss': tensor(1.1415, grad_fn=<AddBackward0>), 'log_vars': OrderedDict([('loss', 1.1415321826934814)]), 'num_samples': 1}
{'loss': tensor(0.8870, grad_fn=<AddBackward0>), 'log_vars': OrderedDict([('loss', 0.887002170085907)]), 'num_samples': 1}
{'loss': tensor(1.1295, grad_fn=<AddBackward0>), 'log_vars': OrderedDict([('loss', 1.1294714212417603)]), 'num_samples': 1}
{'loss': tensor(0.8797, grad_fn=<AddBackward0>), 'log_vars': OrderedDict([('loss', 0.8797056078910828)]), 'num_samples': 1}
{'loss': tensor(0.9411, grad_fn=<AddBackward0>), 'log_vars': OrderedDict([('loss', 0.9410520195960999)]), 'num_samples': 1}
{'loss': tensor(1.1448, grad_fn=<AddBackward0>), 'log_vars': OrderedDict([('loss', 1.1448103189468384)]), 'num_samples': 1}
{'loss': tensor(0

In [9]:
from otx.v2.adapters.torch.mmcv.mmcls.modules.models import SAMImageClassifier
model_2 = SAMImageClassifier(
    backbone=dict(type='otx.OTXEfficientNet', pretrained=True, version='b0'),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='CustomLinearClsHead',
        num_classes=train_dataloader.dataset.num_classes,
        in_channels=1280,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1,)
    ),
)

print(f"Model type: {type(model_2)}")

2023-07-05 13:26:45,819 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
Model type: <class 'otx.v2.adapters.torch.mmcv.mmcls.modules.models.classifiers.sam_classifier.SAMImageClassifier'>


## Training

Users can use each framework's training provided by OTX. (Engine)

- The engine requires the necessary models and DataLoaders for each framework.

In [10]:
# Use Custom Hook & other things -> mmX Runner
mmcls_engine = MMCLSEngine()

output_weight_path = mmcls_engine.train(
    model=model,  # torch.nn.Module
    train_dataloader=train_dataloader,  # torch.utils.data.DataLoader
    work_dir="/home/harimkan/workspace/repo/otx-fork/otx-workspace-test",
    max_epochs=2,
)

print(output_weight_path)

2023-07-05 13:26:49,484 | INFO : Start running, host: harimkan@harimkang-desktop, work_dir: /home/harimkan/workspace/repo/otx-fork/otx-workspace-test
2023-07-05 13:26:49,485 | INFO : Hooks will be executed in the following order:
before_run:
(NORMAL      ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(LOW         ) IterTimerHook                      
 -------------------- 
before_train_iter:
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(NORMAL      ) CheckpointHook                     
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_epoch:
(NORMAL      ) CheckpointHook                     
 -------------------- 
before_val_epoch:
(LOW         ) IterTimerHook                      
 -------------------- 
before_val_iter:
(LOW         ) IterTimerHook                      
 -------------------- 
after_val_iter:
(LOW         ) IterTimerHook                      
 ----------

2023-07-05 13:26:50,207 | INFO : Saving checkpoint at 1 epochs
2023-07-05 13:26:50,968 | INFO : Saving checkpoint at 2 epochs
/home/harimkan/workspace/repo/otx-fork/otx-workspace-test/latest.pth


## OTX AutoEngine (Automation Training API)
OTX provides a more convenient API called AutoEngine.

- It's more convenient for users to use Engine, which provides auto-configuration and the features provided by OTX without having to choose a framework.
- Prepare Dataset & DataLoader + Prepare Model + OTX Recipes + Training + ETC.
- This will make all of the above steps happen automatically. (Auto: Model Selection & build, Dataset Configuration, Training, etc..)

In [12]:
from otx.v2.api.core.engine import AutoEngine

output_dir = "/tmp/OTX-API-test"
data_roots = "/home/harimkan/workspace/repo/otx-fork-3/tests/assets/classification_dataset_class_incremental"
default_config_path = "/home/harimkan/workspace/repo/otx-fork-3/src/otx/v2/configs/classification/mmcls_efficientnet_b0.yaml"

engine = AutoEngine(
    work_dir=output_dir,
    train_data_roots=data_roots,
    config=default_config_path,
)

# Auto-mation training
# output_weight_path = engine.train()
# Customization training
output_weight_path = engine.train(batch_size=2, max_epochs=2)
print(output_weight_path)

2023-07-05 13:27:30,807 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-07-05 13:27:30,826 | INFO : 'in_channels' config in model.head is updated from -1 to 1280
2023-07-05 13:27:30,884 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-07-05 13:27:30,928 | INFO : Start running, host: harimkan@harimkang-desktop, work_dir: /tmp/OTX-API-test
2023-07-05 13:27:30,929 | INFO : Hooks will be executed in the following order:
before_run:
(NORMAL      ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(LOW         ) IterTimerHook                      
 -------------------- 
before_train_iter:
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(NORMAL      ) CheckpointHook                     
(LOW         ) IterTimerHook                      
 -------------------- 
after_t

2023-07-05 13:27:31,607 | INFO : Saving checkpoint at 1 epochs
2023-07-05 13:27:32,308 | INFO : Saving checkpoint at 2 epochs
/tmp/OTX-API-test/latest.pth


In [13]:
# Add validation step
engine = AutoEngine(
    work_dir=output_dir,
    train_data_roots=data_roots,
    val_data_roots=data_roots,
    config=default_config_path,
)

output_weight_path = engine.train(batch_size=2, max_epochs=2)
print(output_weight_path)

2023-07-05 13:27:35,532 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-07-05 13:27:35,551 | INFO : 'in_channels' config in model.head is updated from -1 to 1280
2023-07-05 13:27:35,609 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-07-05 13:27:35,660 | INFO : Start running, host: harimkan@harimkang-desktop, work_dir: /tmp/OTX-API-test
2023-07-05 13:27:35,661 | INFO : Hooks will be executed in the following order:
before_run:
(ABOVE_NORMAL) CustomEvalHook                     
(NORMAL      ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(ABOVE_NORMAL) CustomEvalHook                     
(LOW         ) IterTimerHook                      
 -------------------- 
before_train_iter:
(ABOVE_NORMAL) CustomEvalHook                     
(LOW         ) IterTimerHook                      
 -------------------

/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/mmcv/runner/hooks/evaluation.py:226: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 32/32, 82.2 task/s, elapsed: 0s, ETA:     0s
2023-07-05 13:27:36,724 | INFO : Saving checkpoint at 1 epochs
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 32/32, 83.2 task/s, elapsed: 0s, ETA:     0s
2023-07-05 13:27:37,819 | INFO : Saving checkpoint at 2 epochs
/tmp/OTX-API-test/latest.pth


## Semi-SL Example

In [14]:
from otx.v2.adapters.torch.mmcv.mmcls import Dataset
dataset = Dataset(
    train_data_roots="/home/harimkan/workspace/datasets/otx_cls_dataset/cifar10@4_0/train_data",
    val_data_roots="/home/harimkan/workspace/datasets/otx_cls_dataset/cifar10@4_0/val_data",
    test_data_roots="/home/harimkan/workspace/datasets/otx_cls_dataset/cifar10@4_0/val_data",
    unlabeled_data_roots="/home/harimkan/workspace/datasets/otx_cls_dataset/cifar10@4_0/unlabel_data"
)

In [15]:
config = "/home/harimkan/workspace/repo/otx-fork/otx-workspace-test/otx-workspace-CLASSIFICATION/semisl/data_pipeline.py"

train_dataloader = dataset.train_dataloader(config=config)


[*] Detected dataset format: imagenet
[*] Detected task type: CLASSIFICATION


[*] Semisupervised training type detected with unlabeled data: /home/harimkan/workspace/datasets/otx_cls_dataset/cifar10@4_0/unlabel_data
2023-07-05 13:27:48,744 | INFO : possible max iterations = 40


In [16]:
dataset.train_type

<TrainType.Semisupervised: 'Semisupervised'>

In [17]:
from otx.v2.adapters.torch.mmcv.mmcls import build_model_from_config
semi_sl_model = dict(
    model=dict(
        type='SemiSLClassifier',
        backbone=dict(
            type='otx.OTXEfficientNet',
            pretrained=True,
            version='b0'),
        neck=dict(
            type='GlobalAveragePooling'),
        head=dict(
            type='SemiLinearClsHead',
            num_classes=1000,
            in_channels=1280,
            loss=dict(
                type='CrossEntropyLoss',
                loss_weight=1.0),
            topk=(1, 5)),
        task='classification',
        pretrained=None)
    )

model = build_model_from_config(
    config=semi_sl_model,
    num_classes=10,
)

2023-07-05 13:27:57,514 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip


In [18]:
from otx.v2.adapters.torch.mmcv.mmcls.engine import MMCLSEngine
mmcls_engine = MMCLSEngine()

output = mmcls_engine.train(
    model=model,
    train_dataloader=train_dataloader,
    work_dir="/home/harimkan/workspace/repo/otx-fork/otx-workspace-test",
    max_epochs=3,
)

print(output)

2023-07-05 13:28:04,797 | INFO : Start running, host: harimkan@harimkang-desktop, work_dir: /home/harimkan/workspace/repo/otx-fork/otx-workspace-test
2023-07-05 13:28:04,798 | INFO : Hooks will be executed in the following order:
before_run:
(NORMAL      ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(LOW         ) IterTimerHook                      
 -------------------- 
before_train_iter:
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(NORMAL      ) CheckpointHook                     
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_epoch:
(NORMAL      ) CheckpointHook                     
 -------------------- 
before_val_epoch:
(LOW         ) IterTimerHook                      
 -------------------- 
before_val_iter:
(LOW         ) IterTimerHook                      
 -------------------- 
after_val_iter:
(LOW         ) IterTimerHook                      
 ----------

2023-07-05 13:28:07,346 | INFO : Saving checkpoint at 1 epochs
2023-07-05 13:28:09,891 | INFO : Saving checkpoint at 2 epochs
2023-07-05 13:28:12,419 | INFO : Saving checkpoint at 3 epochs
/home/harimkan/workspace/repo/otx-fork/otx-workspace-test/latest.pth


In [19]:
custom_hooks=[
    dict(
        type='ModelEmaV2Hook',
        priority='ABOVE_NORMAL'
    ),
    dict(type='SemiSLClsHook')
]


output = mmcls_engine.train(
    model=model,
    train_dataloader=train_dataloader,
    custom_hooks=custom_hooks,
    work_dir="/home/harimkan/workspace/repo/otx-fork/otx-workspace-test",
    max_epochs=3,
)

print(output)

2023-07-05 13:28:17,538 | INFO : Start running, host: harimkan@harimkang-desktop, work_dir: /home/harimkan/workspace/repo/otx-fork/otx-workspace-test
2023-07-05 13:28:17,539 | INFO : Hooks will be executed in the following order:
before_run:
(ABOVE_NORMAL) ModelEmaV2Hook                     
(NORMAL      ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(ABOVE_NORMAL) ModelEmaV2Hook                     
(LOW         ) IterTimerHook                      
 -------------------- 
before_train_iter:
(NORMAL      ) SemiSLClsHook                      
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) ModelEmaV2Hook                     
(NORMAL      ) CheckpointHook                     
(NORMAL      ) SemiSLClsHook                      
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_epoch:
(NORMAL      ) CheckpointHook                     
(NORMAL      ) SemiSLClsHook 

2023-07-05 13:28:20,064 | INFO : Saving checkpoint at 1 epochs
2023-07-05 13:28:22,631 | INFO : Saving checkpoint at 2 epochs
2023-07-05 13:28:25,178 | INFO : Saving checkpoint at 3 epochs
/home/harimkan/workspace/repo/otx-fork/otx-workspace-test/latest.pth
